<a href="https://colab.research.google.com/github/akash02ita/CPSC-599.6-DL-for-Vision/blob/main/a2/DL4Vision_A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2
In this assignment, we will use a multi-layer perceptron network to build an image classifier for single digits. We will be using a public dataset for model development. The dataset we will be using is the MNIST digit dataset. The dataset contains 10 classes, where class `i` contains images of digit `i`.

In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import torch.optim as optim

####1. Create train_data and test_dataset objects using the MNIST digit dataset from torchvision.datasets module. (5 points)

In [2]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = datasets.MNIST(root='data', train = True, transform=transform, download = True)
test_dataset = datasets.MNIST(root='data', train = False, transform=transform, download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



####2. Use the [random_split](https://pytorch.org/docs/stable/data.html#torch.utils.data.random_split) method to split the `train_data` into `train_dataset` (50000 images) and `validation_dataset` dataset (10000 images). (5 points)

In [3]:
print(len(train_data))
train_dataset, validation_dataset = random_split(train_data, [50000, 10000], generator=torch.Generator().manual_seed(42))
len(train_dataset), len(validation_dataset)

60000


(50000, 10000)

#### 3. Create dataloader objects for `train_dataset`, `validation_dataset`, and `test_dataset`. (5 points)

In [4]:
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
validation_dataloader = DataLoader(validation_dataset, batch_size = 32, shuffle = True)
type(train_dataloader), len(train_dataloader), len(train_dataloader.dataset)

(torch.utils.data.dataloader.DataLoader, 1563, 50000)

In [5]:
for images, labels in train_dataloader:
  print(type(images), type(labels))
  print(images.shape, labels.shape)
  print(images[0].numpy().shape)
  break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([32, 1, 28, 28]) torch.Size([32])
(1, 28, 28)


#### 4. Develop an MLP model for classifying MNIST images. The developed model should have four hidden layers of 256, 128, 64, and 32 neurons. Each hidden layer should be followed with a ReLU unit and a Dropout layer (p=0.2).  (15 points)

In [6]:
flattened_image_shape = 1*28*28
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(flattened_image_shape, 256, bias=True),
    nn.ReLU(),
    nn.Linear(256, 128, bias=True),
    nn.ReLU(),
    nn.Linear(128, 64, bias=True),
    nn.ReLU(),
    nn.Linear(64, 32, bias=True),
    nn.Dropout(p=0.2),
    nn.Linear(32, 10, bias=True)
)

#### 5. Define the components needed for training a deep learning model. (10 points)



In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device) # returns model on the device

cuda


#### 6. Write the training loop and train the model for 100 epochs. Print the training and validation accuracy and loss for each epoch. (35 points)

In [9]:
model.train()
epochs = 100
for epoch in range(epochs):
  for current_data, current_dataloader  in [("TRAIN", train_dataloader), ("VALIDATION", validation_dataloader)]:
    total_loss = 0
    total = 0
    correct = 0
    for images, labels in current_dataloader:
      optimizer.zero_grad()
      # data also needs to move to same device (in this case gpu)
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      loss = criterion(outputs, labels)
      if current_data == "TRAIN": # only when using training dataloader
        loss.backward()
        optimizer.step()
      total_loss += loss.item() * images.size(0)
      total += images.size(0)
      _, preds = torch.max(outputs, 1) # preds is index of maximum found in each vector ouput \element of ouputs
      correct += (preds == labels).sum().item()
    accuracy = correct / total
    loss = total_loss / total
    print(f"{epoch} {current_data}: Accuracy = {accuracy}, Loss = {loss}")


0 TRAIN: Accuracy = 0.78302, Loss = 0.6430898488974571
0 VALIDATION: Accuracy = 0.9431, Loss = 0.19452546980082988
1 TRAIN: Accuracy = 0.95806, Loss = 0.14460179238945245
1 VALIDATION: Accuracy = 0.9544, Loss = 0.15385555286109448
2 TRAIN: Accuracy = 0.9709, Loss = 0.09684254695415496
2 VALIDATION: Accuracy = 0.966, Loss = 0.11985207977965474
3 TRAIN: Accuracy = 0.97754, Loss = 0.07377695428997279
3 VALIDATION: Accuracy = 0.9611, Loss = 0.13645503444075585
4 TRAIN: Accuracy = 0.983, Loss = 0.05704037877427414
4 VALIDATION: Accuracy = 0.9655, Loss = 0.12336133925598115
5 TRAIN: Accuracy = 0.98542, Loss = 0.046785511205969375
5 VALIDATION: Accuracy = 0.9733, Loss = 0.10106248338986189
6 TRAIN: Accuracy = 0.9875, Loss = 0.03911058475411497
6 VALIDATION: Accuracy = 0.9728, Loss = 0.1110442432298325
7 TRAIN: Accuracy = 0.9904, Loss = 0.03130052413311787
7 VALIDATION: Accuracy = 0.9708, Loss = 0.12459521320571658
8 TRAIN: Accuracy = 0.99192, Loss = 0.026174164674571947
8 VALIDATION: Accuracy

####6. Test the model using the `test_dataset`, and report accuracy and loss. (10 points)

In [16]:
model.eval()
total_loss = 0
total = 0
correct = 0
for images, labels in test_dataloader:
  images, labels = images.to(device), labels.to(device)
  with torch.no_grad():
    outputs = model(images)
    loss = criterion(outputs, labels)
    total_loss += loss.item() * images.size(0)
    total += images.size(0)
    _, preds = torch.max(outputs, 1) # preds is index of maximum found in each vector ouput \element of ouputs
    correct += (preds == labels).sum().item()
accuracy = correct / total
loss = total_loss / total
print(f"TEST: Accuracy = {accuracy}, Loss = {loss}")

TEST: Accuracy = 0.9844, Loss = 0.13774921294286466


#### 8. We would like to see how accurate the trained model is when applied to a set of images of digits with slight differences. The image of a digit in the MNIST dataset has a black background (0 value for pixel values). This might be like writing with white chalk on a blackboard. We make slight changes in the test datasets by applying a simple change `image = 1 - image`. In other words, we invert the pixel intensity values. The resulting images resemble a digit written with a black marker on a whiteboard. Test the model using the updated test_dataset, and report your observations regarding model performance. How would you change your pipeline if you redo this experiment? (15 points)